In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# system size


In [ ]:
# initialization


# energy



In [ ]:
# update of Jastrow factor



In [ ]:
# update of determinant


In [ ]:
# energy



In [ ]:
# variation
